In [1]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

In [14]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#apro il file playlist che contiene il nome dell'utente e il link alla sua playlist
#metto i dati nella lista people
people = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people.append(row)
        break

people2 = list()
with open('viral.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people2.append(row)
        break

aux = {}
aux2 = {}

#prendo il link della playlist da person 
playlist = spotify.playlist(people[0]["Link"])
#dal link estraggo l'id della playlist
songs = spotify.playlist_items(playlist["id"]) 
    
for i in range (10):
    isrc = songs["items"][i]["track"]["external_ids"]["isrc"]
    
    track_uri = songs["items"][i]["track"]["uri"]
    track_name = songs["items"][i]["track"]["name"]
    
    artist_name = songs["items"][i]["track"]["artists"][i]["name"]
    artist_genres = spotify.artist(songs["items"][i]["track"]["artists"][i]["id"])['genres'] #generi dell'artista
    album = spotify.album(songs["items"][i]["track"]["album"]["external_urls"]["spotify"])
    data = album["release_date"]
        
    aux[i] = {"isrc":isrc,"uri":track_uri,"titolo":track_name,"artista":artist_name,"pubblicazione":data, "generi_artista":artist_genres}


#prendo il link della playlist da person 
playlist = spotify.playlist(people2[0]["Link"])
#dal link estraggo l'id della playlist
songs = spotify.playlist_items(playlist["id"]) 
    
for i in range (10):
    isrc = songs["items"][i]["track"]["external_ids"]["isrc"]

    track_uri = songs["items"][i]["track"]["uri"]
    track_name = songs["items"][i]["track"]["name"]
    artist_genres = spotify.artist(songs["items"][i]["track"]["artists"][i]["id"])['genres'] #generi dell'artista

    artist_name = songs["items"][i]["track"]["artists"][0]["name"]

    album = spotify.album(songs["items"][i]["track"]["album"]["external_urls"]["spotify"])
    data = album["release_date"]
    
    aux2[i] = {"isrc":isrc,"uri":track_uri,"titolo":track_name,"artista":artist_name,"pubblicazione":data, "generi_artista":artist_genres}

with open('10-global.csv', 'w+', newline='', encoding='utf-16') as file:
    writer = csv.DictWriter(file,fieldnames=["isrc","uri","titolo","artista","pubblicazione", "generi_artista"])
    #scrive Id Label nel file csv come intestazione
    writer.writeheader()
    #scrive id affiancato da nome nel file
    for i in range (10) :
        writer.writerow(aux[i])

with open('10-viral.csv', 'w+', newline='', encoding='utf-16') as file:
    writer = csv.DictWriter(file,fieldnames=["isrc","uri","titolo","artista","pubblicazione", "generi_artista"])
    #scrive Id Label nel file csv come intestazione
    writer.writeheader()
    #scrive id affiancato da nome nel file
    for i in range (10) :
        writer.writerow(aux2[i])

['pov: indie', 'psychedelic pop']


'with open(\'10-global.csv\', \'w+\', newline=\'\', encoding=\'utf-16\') as file:\n    writer = csv.DictWriter(file,fieldnames=["isrc","uri","titolo","artista","pubblicazione", "nazioneArtista"])\n    #scrive Id Label nel file csv come intestazione\n    writer.writeheader()\n    #scrive id affiancato da nome nel file\n    for i in range (10) :\n        writer.writerow(aux[i])\n\nwith open(\'10-viral.csv\', \'w+\', newline=\'\', encoding=\'utf-16\') as file:\n    writer = csv.DictWriter(file,fieldnames=["isrc","uri","titolo","artista","pubblicazione", "nazioneArtista"])\n    #scrive Id Label nel file csv come intestazione\n    writer.writeheader()\n    #scrive id affiancato da nome nel file\n    for i in range (10) :\n        writer.writerow(aux2[i])'

In [10]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))


people = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people.append(row)

#questo è il mio dataset a ogni indice corrisponde una lista di codici ISRC (30 per playlist tranne in rari casi)
dataset = list()

#prendo il link della playlist dalla lista people creata prima
#dal link estraggo l'id della playlist
#dall'id della playlist estraggo tutti i brani da ogni brano estraggo il codice ISRC
for person in people:

    #prendo il link della playlist da person 
    playlist = spotify.playlist(person["Link"])
    #dal link estraggo l'id della playlist
    songs = spotify.playlist_items(playlist["id"]) 

    #insieme ausiliario in cui metto tutti gli ISRC di una singola playlist
    aux = set()

    #estraggo da ogni canzone il codice ISRC e lo inserisco nell'insieme aux
    for i in range (len(songs["items"])):
        aux.add(songs["items"][i]["track"]["external_ids"]["isrc"])
    
    #appendo al mio dataset gli ISRC estratti per ogni playlist
    #indice 0 avrò codici ISRC della playlist dell'utente 0 che in realtà è gia 
    #relazionato con il suo nome
    dataset.append(aux)

#----FINE----API-SPOTIFY----FINE----

#n risulta uguale alla lunghezza del nostro dataset quindi al numero di utenti
#per il quali siamo andati ad estrarre le canzoni dalla playlist
n = len(dataset)

centrality = {}
for x in range (1,n) : 
        affinity = len(dataset[0].intersection(dataset[x]))
        centrality[people[x]["Name"]] = affinity

#print(centrality)

centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

header = ['Paese', 'Centralità']
with open('global-centrality.csv', 'w+', newline='', encoding='utf-16') as file:
    #writer = csv.DictWriter(file,fieldnames=["Paese","Centralità"])
    writer = csv.writer(file)
    #scrive Paese Centralità nel file csv come intestazione
    writer.writerow(header)
    #scrive id affiancato da nome nel file
    for i in range (len(centrality)) :
        writer.writerow(centrality[i])

grafo = nx.Graph()

for i in range (1,len(people)) :
        #writer.writerow({"Id":i,"Label":people[i]["Name"]})
        grafo.add_node(i,label=people[i]["Name"],latitudine=float(people[i]["latitudine"]),longitudine=float(people[i]["longitudine"]))

#dato che trattiamo le canzoni estratte come insiemi a coppie di utenti
#intersechiamo questi insiemi ed estraiamo il numero di elementi
#ovvero le canzoni che hanno in comune ovvero il loro grado di affinità
for x in range (1,n) : 
    for y in range (x+1,n) :
        #intersezione degli insiemi di canzoni tra utenti
        affinity = len(dataset[x].intersection(dataset[y]))

        #se l'intersezione non è vuota vuol dire che ho un link valido
        #pertanto aggiungo gli id degli utenti e il numero di canzoni
        #che hanno in comune
        if affinity :
            grafo.add_edge(x,y,weight=affinity)
            #edges.append({"Source":x, "Target":y, "Weight":affinity})

nx.write_gexf(grafo,'global.gexf')
#nx.write_gpickle(grafo,'global.pkl')
with open('global1.pkl', 'wb') as f:
    pickle.dump(grafo, f)

In [11]:
people = list()
with open('viral.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people.append(row)

#questo è il mio dataset a ogni indice corrisponde una lista di codici ISRC (30 per playlist tranne in rari casi)
dataset = list()

#prendo il link della playlist dalla lista people creata prima
#dal link estraggo l'id della playlist
#dall'id della playlist estraggo tutti i brani da ogni brano estraggo il codice ISRC
for person in people:

    #prendo il link della playlist da person 
    playlist = spotify.playlist(person["Link"])
    #dal link estraggo l'id della playlist
    songs = spotify.playlist_items(playlist["id"]) 

    #insieme ausiliario in cui metto tutti gli ISRC di una singola playlist
    aux = set()

    #estraggo da ogni canzone il codice ISRC e lo inserisco nell'insieme aux
    for i in range (len(songs["items"])):
        aux.add(songs["items"][i]["track"]["uri"])
        #aux.add(songs["items"][i]["track"]["external_ids"]["isrc"])
    
    #appendo al mio dataset gli ISRC estratti per ogni playlist
    #indice 0 avrò codici ISRC della playlist dell'utente 0 che in realtà è gia 
    #relazionato con il suo nome
    dataset.append(aux)

#----FINE----API-SPOTIFY----FINE----

#n risulta uguale alla lunghezza del nostro dataset quindi al numero di utenti
#per il quali siamo andati ad estrarre le canzoni dalla playlist
n = len(dataset)

centrality = {}
for x in range (1,n) : 
        affinity = len(dataset[0].intersection(dataset[x]))
        centrality[people[x]["Name"]] = affinity

#print(centrality)

centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

header = ['Paese', 'Centralità']
with open('viral-centrality.csv', 'w+', newline='', encoding='utf-16') as file:
    #writer = csv.DictWriter(file,fieldnames=["Paese","Centralità"])
    writer = csv.writer(file)
    #scrive Paese Centralità nel file csv come intestazione
    writer.writerow(header)
    #scrive id affiancato da nome nel file
    for i in range (len(centrality)) :
        writer.writerow(centrality[i])

grafo = nx.Graph()

for i in range (1,len(people)) :
        #writer.writerow({"Id":i,"Label":people[i]["Name"]})
        grafo.add_node(i,label=people[i]["Name"],latitudine=float(people[i]["latitudine"]),longitudine=float(people[i]["longitudine"]))

#dato che trattiamo le canzoni estratte come insiemi a coppie di utenti
#intersechiamo questi insiemi ed estraiamo il numero di elementi
#ovvero le canzoni che hanno in comune ovvero il loro grado di affinità
for x in range (1,n) : 
    for y in range (x+1,n) :
        #intersezione degli insiemi di canzoni tra utenti
        affinity = len(dataset[x].intersection(dataset[y]))

        #se l'intersezione non è vuota vuol dire che ho un link valido
        #pertanto aggiungo gli id degli utenti e il numero di canzoni
        #che hanno in comune
        if affinity :
            grafo.add_edge(x,y,weight=affinity)
            #edges.append({"Source":x, "Target":y, "Weight":affinity})

nx.write_gexf(grafo,'viral.gexf')
#nx.write_gpickle(grafo,'viral.pkl')
with open('viral1.pkl', 'wb') as f:
    pickle.dump(grafo, f)

In [21]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

# Lista dei paesi da esaminare
countries = ['US', 'GB', 'DE', 'IT', 'FR', 'ES']  # Aggiungi o rimuovi codici di paesi secondo necessità

# Dizionario per conservare i dati
top_tracks = {}

# Raccogli i dati delle top tracks per ciascun paese
for country in countries:
    top_tracks_country = spotify.playlist_tracks('spotify:playlist:37i9dQZEVXbMDoHDwVN2tF', market=country)  # Usa la playlist "Global Top 50"
    top_tracks[country] = [(track['track']['name'], track['track']['artists'][0]['name']) for track in top_tracks_country['items']]

# Converti il dizionario in un DataFrame per un'analisi più facile
df_tracks = pd.DataFrame(columns=['Country', 'Track', 'Artist'])
for country, tracks in top_tracks.items():
    for track in tracks:
        df_tracks = pd.concat([df_tracks, pd.DataFrame({'Country': [country], 'Track': [track[0]], 'Artist': [track[1]]})], ignore_index=True)

# Analizza i dati
# Ad esempio, contiamo quante volte ogni brano appare in diverse liste nazionali
track_counts = df_tracks.groupby(['Track', 'Artist']).Country.nunique().reset_index(name='Count')
top_international_tracks = track_counts[track_counts.Count > 1].sort_values('Count', ascending=False)

print(top_international_tracks)

                                                Track  \
0                                           As It Was   
37                                    Training Season   
27                        Praise Jah In The Moonlight   
28                                              QLONA   
29                                             Saturn   
30                                    Scared To Start   
31                Seven (feat. Latto) (Explicit Ver.)   
32                                            Starboy   
33                                       Stick Season   
34                                          Strangers   
35                                     TEXAS HOLD 'EM   
36                                   The Night We Met   
38                                          Unwritten   
1                                              Austin   
39                                              Water   
40  What Was I Made For? [From The Motion Picture ...   
41                             